In [2]:
import pandas as pd
import math
clinical = pd.read_csv("ADSP_PHC_COGN_17Apr2024.csv").rename(columns={"PHASE":"Phase"})
#this file is the metadata file that one can get from downloading MRI images from ADNI
img = pd.read_csv("metafinal1_4_20_2024.csv")
comb = pd.read_csv("DXSUM_PDXCONV_17Apr2024.csv")[["RID", "PTID" , "PHASE"]].rename(columns={"PHASE":"Phase"})

C:\Users\Adya2\AppData\Local\Temp\ipykernel_28532\4083515485.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
def read_diagnose(file_path: str = 'DXSUM_PDXCONV_17Apr2024.csv', verbose=False):
    diagnostic_summary = pd.read_csv(file_path, index_col='PTID')
    diagnostic_summary = diagnostic_summary.sort_values(by=["update_stamp"], ascending=True)
    diagnostic_dict: dict = {}
    for key, data in diagnostic_summary.iterrows():
        phase: str = data['PHASE']
        diagnosis: float = -1.
        
        
        if phase == "ADNI2" or phase == "ADNIGO" or phase == "ADNI1" or phase =="ADNI4" or phase=="ADNI3":
            diagnosis = data['DIAGNOSIS']
            
        else:
            print(f"ERROR: Not recognized study phase {phase}")
            exit(1)
        if not math.isnan(diagnosis):
            diagnostic_dict[key] = diagnosis

    return diagnostic_dict
def print_diagnostic_dict_summary(diagnostic_dict: dict):
    print(f"Number of diagnosed patients: {len(diagnostic_dict.items())}\n")
    n_CN = 0
    n_MCI = 0
    n_AD = 0
    for (key, data) in diagnostic_dict.items():
        if data == 1:
            n_CN += 1
        if data == 2:
            n_MCI += 1
        if data == 3:
            n_AD += 1
        
    print(f"Number of CN patients: {n_CN}\n",f"Number of MCI patients: {n_MCI}\n",f"Number of AD patients: {n_AD}\n")


Here, we are just trying to see what the data looks like and what we can do with it

In [7]:
img.head()
class_counts=img['Group'].value_counts()
class_counts


Group
CN     1210
MCI     946
AD      458
Name: count, dtype: int64

In [8]:
d = read_diagnose()
print_diagnostic_dict_summary(d)

Number of diagnosed patients: 3037

Number of CN patients: 1127
 Number of MCI patients: 1013
 Number of AD patients: 897



We will convert this dictionary into a dataframe called new which has only two columns which is basically the subject ID and the group

In [9]:
comb.head()
#comb ==DXSUM 17APR2024

,RID,PTID,Phase
0,2,011_S_0002,ADNI1
1,3,011_S_0003,ADNI1
2,5,011_S_0005,ADNI1
3,8,011_S_0008,ADNI1
4,7,022_S_0007,ADNI1


In [10]:
new = pd.DataFrame.from_dict(d, orient='index').reset_index()
new.columns

Index(['index', 0], dtype='object')

In [11]:
clinical.head()
clinical["year"] = clinical["EXAMDATE"].str[:4]
clinical["Subject"] = clinical["SUBJECT_KEY"].str.replace("ADNI_", "").str.replace("s", "S")
c = comb.merge(clinical, on = ["RID", "Phase"])

In clinical, we got a new column having just the year of examination.
The subject column was rewritten 
c now stores the ADNI_PHC_COGN + DXSUM File

In [12]:
comb.head()

,RID,PTID,Phase
0,2,011_S_0002,ADNI1
1,3,011_S_0003,ADNI1
2,5,011_S_0005,ADNI1
3,8,011_S_0008,ADNI1
4,7,022_S_0007,ADNI1


In [9]:
clinical.head()

,RID,SUBJECT_KEY,Phase,VISCODE,VISCODE2,EXAMDATE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,AGE,DX,PHC_MEM,PHC_EXF,PHC_LAN,PHC_VSP,update_stamp,year,Subject
0,2,ADNI_011_S_0002,ADNI2,v06,m72,2011-09-19,1,16.0,2,5,80.4682,1.0,0.090,0.002,0.460,0.264,2021-04-14 13:05:55.0,2011,011_S_0002
1,2,ADNI_011_S_0002,ADNI2,v21,m96,2013-09-09,1,16.0,2,5,82.4422,1.0,0.288,-0.495,0.460,NaN,2021-04-14 13:05:56.0,2013,011_S_0002
2,2,ADNI_011_S_0002,ADNI1,m36,m36,2008-08-27,1,16.0,2,5,77.4073,1.0,0.344,0.419,0.535,NaN,2021-04-14 13:05:56.0,2008,011_S_0002
3,2,ADNI_011_S_0002,ADNI2,v41,m120,2015-09-22,1,16.0,2,5,84.4764,1.0,0.382,-0.549,0.595,-0.333,2021-04-14 13:05:56.0,2015,011_S_0002
4,2,ADNI_011_S_0002,ADNIGO,m60,m60,2010-09-22,1,16.0,2,5,79.4771,1.0,0.101,0.066,0.620,0.264,2021-04-14 13:05:56.0,2010,011_S_0002


In [ ]:
c

In [13]:
c = c.drop("Subject", axis =1)
c = c.rename(columns = {"PTID":"Subject"})
c

,RID,Subject,Phase,SUBJECT_KEY,VISCODE,VISCODE2,EXAMDATE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,AGE,DX,PHC_MEM,PHC_EXF,PHC_LAN,PHC_VSP,update_stamp,year
0,2,011_S_0002,ADNI1,ADNI_011_S_0002,m36,m36,2008-08-27,1,16.0,2,5,77.4073,1.0,0.344,0.419,0.535,NaN,2021-04-14 13:05:56.0,2008
1,2,011_S_0002,ADNI1,ADNI_011_S_0002,bl,bl,2005-09-08,1,16.0,2,5,74.4394,1.0,0.277,0.254,0.293,NaN,2021-04-14 13:05:56.0,2005
2,2,011_S_0002,ADNI1,ADNI_011_S_0002,m06,m06,2006-03-06,1,16.0,2,5,74.9295,1.0,0.237,0.278,0.535,-0.333,2021-04-14 13:05:56.0,2006
3,3,011_S_0003,ADNI1,G-ADNI-AN000439,m12,m12,2006-09-12,1,18.0,2,5,82.3655,3.0,-0.973,-1.036,-0.828,0.128,2021-04-14 13:05:56.0,2006
4,3,011_S_0003,ADNI1,G-ADNI-AN000439,bl,bl,2005-09-12,1,18.0,2,5,81.3662,3.0,-1.068,-0.670,-0.516,-0.237,2021-04-14 13:05:56.0,2005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50415,6220,109_S_6220,ADNI3,NaN,bl,bl,2019-01-17,2,12.0,2,4,74.7953,2.0,0.403,0.644,0.555,NaN,2021-04-14 13:06:12.0,2019
50416,6668,019_S_6668,ADNI3,NaN,bl,bl,2019-02-06,2,15.0,2,5,75.6852,2.0,-0.512,-0.149,0.821,-0.094,2021-04-14 13:06:13.0,2019
50417,6251,114_S_6251,ADNI3,NaN,y1,m12,2019-08-13,2,16.0,2,5,67.0308,1.0,1.127,1.506,1.561,NaN,2021-04-14 13:06:12.0,2019
50418,6251,114_S_6251,ADNI3,NaN,bl,bl,2018-06-06,2,16.0,2,5,65.8453,1.0,1.317,1.117,1.035,0.483,2021-04-14 13:06:12.0,2018


In [14]:
img["year"] = img["Acq Date"].str[5:].str.replace("/", "").str.replace("-","")
img = img.replace(["CN", "MCI", "AD"], [ 0, 1, 2])
img

C:\Users\Adya2\AppData\Local\Temp\ipykernel_28532\3744868513.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  img = img.replace(["CN", "MCI", "AD"], [ 0, 1, 2])


,Subject,Unnamed: 1,Sex,Group,Age,Modality,Description,Image ID,Acq Date,year
0,002_S_0295,NaN,M,0,84.9,MRI,B1-Calibration Body,13710,09-01-2005,2005
1,002_S_0295,NaN,M,0,90.0,MRI,Axial T2-Star,238619,12-08-2005,2005
2,002_S_0413,NaN,F,0,76.4,MRI,B1-Calibration PA,14427,12-09-2005,2005
3,002_S_0413,NaN,F,0,81.5,MRI,Axial T2-FLAIR,240806,12-12-2005,2005
4,002_S_0413,NaN,F,0,87.5,MRI,Perfusion_Weighted,863054,02-01-2006,2006
...,...,...,...,...,...,...,...,...,...,...
2609,941_S_7074,NaN,M,0,70.9,MRI,Axial 3TE T2 STAR,1588329,9/30/2020,2020
2610,941_S_7074,NaN,M,0,72.2,MRI,HighResHippocampus (MSV21),10269461,9/30/2020,2020
2611,941_S_7085,NaN,F,1,69.7,MRI,Axial 3TE T2 STAR,1600178,9/30/2021,2021
2612,941_S_7087,NaN,M,0,66.9,MRI,Axial 3TE T2 STAR,1591319,9/30/2021,2021


In [15]:
new.columns
new.head()

,index,0
0,011_S_0002,2.0
1,011_S_0003,3.0
2,011_S_0005,1.0
3,011_S_0008,2.0
4,022_S_0007,3.0


In [16]:
new.columns
new = new.rename(columns = {"index":"Subject", 0:"GroupN"})
new['GroupN'] = new['GroupN'].astype(int) -1
c['DX']=c['DX']-1
#img.columns
new

,Subject,GroupN
0,011_S_0002,1
1,011_S_0003,2
2,011_S_0005,0
3,011_S_0008,1
4,022_S_0007,2
...,...,...
3032,037_S_10063,0
3033,033_S_10099,0
3034,135_S_10097,1
3035,941_S_10103,1


In [17]:
c["Subject"] = c["Subject"].astype(str)
img["Subject"] = img["Subject"].astype(str)
new["Subject"] = new["Subject"].astype(str)




In [18]:
m = new.merge(c, on = "Subject" ,how = "outer").merge(img, on = "Subject", how = "outer")
m
m[["GroupN", "DX", "Group"]]


,GroupN,DX,Group
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0
...,...,...,...
78740,0.0,NaN,0.0
78741,1.0,NaN,1.0
78742,0.0,NaN,0.0
78743,0.0,NaN,NaN


Now we will drop duplicates on the basis of particular rows

In [19]:
m = m[["Subject", "GroupN", "Group", "DX","Phase"]].drop_duplicates()


In [20]:
m = m.dropna(subset = ["GroupN", "Group", "DX","Phase"], how="all").drop_duplicates()
#this basically drops those rows in which all the above columns have null values and then drops duplicates again

In [21]:
m.loc[m["DX"].isna() & m["Group"].isna(), "Group"] = m.loc[m["DX"].isna() & m["Group"].isna(), "GroupN"]
m.loc[m["DX"].isna() & m["Group"].isna(), "DX"] = m.loc[m["DX"].isna() & m["Group"].isna(), "GroupN"]
m1 = m[m["GroupN"] == m["Group"]]
m3 = m[m["GroupN"] == m["DX"]]
m4 = m[m["Group"] == m["DX"]]
m2 = m1[m1["Group"] == m1["DX"]]
m1 = m1[["Subject", "GroupN", "Group", "DX", "Phase"]]
m1

,Subject,GroupN,Group,DX,Phase
0,002_S_0295,0.0,0.0,0.0,ADNI1
72,002_S_0295,0.0,0.0,0.0,ADNI2
80,002_S_0413,0.0,0.0,0.0,ADNI1
188,002_S_0413,0.0,0.0,0.0,ADNI2
263,002_S_0413,0.0,0.0,0.0,ADNI3
...,...,...,...,...,...
78739,941_S_7074,0.0,0.0,NaN,NaN
78741,941_S_7085,1.0,1.0,NaN,NaN
78742,941_S_7087,0.0,0.0,NaN,NaN
78743,941_S_7091,0.0,0.0,NaN,NaN


In [22]:
m1.loc[m1["DX"].isna(), "DX"] = m1.loc[m1["DX"].isna(), "Group"]

The above code replaces those rows

In [23]:
m3 = m3[["Subject", "GroupN", "Group", "DX", "Phase"]]
m3
m3.loc[m3["Group"].isna(), "Group"] = m3.loc[m3["Group"].isna(), "GroupN"]

In the code below, we assign the 5 columns only to m4

In [24]:
m4 = m4[["Subject", "GroupN", "Group", "DX", "Phase"]]
m4

,Subject,GroupN,Group,DX,Phase
0,002_S_0295,0.0,0.0,0.0,ADNI1
72,002_S_0295,0.0,0.0,0.0,ADNI2
80,002_S_0413,0.0,0.0,0.0,ADNI1
188,002_S_0413,0.0,0.0,0.0,ADNI2
263,002_S_0413,0.0,0.0,0.0,ADNI3
...,...,...,...,...,...
78703,941_S_6575,1.0,0.0,0.0,ADNI3
78713,941_S_6580,0.0,0.0,0.0,ADNI3
78723,941_S_6581,0.0,0.0,0.0,ADNI3
78728,941_S_6803,1.0,1.0,1.0,ADNI3


In [25]:
m4[m4["GroupN"] != m4["DX"]]
m2[["Subject", "GroupN", "Group", "DX", "Phase"]]
m5 = pd.concat([m1,m3,m4])
i = m5[m5["Group"] == m5["GroupN"]]
i = i[i["Group"] == i["DX"]]
i = i.drop_duplicates()
i

,Subject,GroupN,Group,DX,Phase
0,002_S_0295,0.0,0.0,0.0,ADNI1
72,002_S_0295,0.0,0.0,0.0,ADNI2
80,002_S_0413,0.0,0.0,0.0,ADNI1
188,002_S_0413,0.0,0.0,0.0,ADNI2
263,002_S_0413,0.0,0.0,0.0,ADNI3
...,...,...,...,...,...
78553,941_S_5124,0.0,0.0,0.0,ADNI3
78554,941_S_5133,0.0,0.0,0.0,ADNI2
78555,941_S_5193,0.0,0.0,0.0,ADNI2
78567,941_S_5193,0.0,0.0,0.0,ADNI3


In [26]:
i[["Subject", "Group", "Phase"]].to_csv("ground_truth.csv")


In [27]:
diag = pd.read_csv("ground_truth.csv").drop("Unnamed: 0", axis=1)

In [28]:
diag

,Subject,Group,Phase
0,002_S_0295,0.0,ADNI1
1,002_S_0295,0.0,ADNI2
2,002_S_0413,0.0,ADNI1
3,002_S_0413,0.0,ADNI2
4,002_S_0413,0.0,ADNI3
...,...,...,...
3189,941_S_5124,0.0,ADNI3
3190,941_S_5133,0.0,ADNI2
3191,941_S_5193,0.0,ADNI2
3192,941_S_5193,0.0,ADNI3


In [30]:
demo = pd.read_csv("PTDEMOG_17Apr2024.csv").rename(columns={"PHASE":"Phase"})

In [29]:
neuro = pd.read_csv("NEUROEXM_17Apr2024.csv").rename(columns={"PHASE":"Phase"})

In [31]:
clinical = pd.read_csv("ADSP_PHC_COGN_17Apr2024.csv").rename(columns={"PHASE":"Phase"})

In [33]:
clinical.head()

,RID,SUBJECT_KEY,Phase,VISCODE,VISCODE2,EXAMDATE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,AGE,DX,PHC_MEM,PHC_EXF,PHC_LAN,PHC_VSP,update_stamp
0,2,ADNI_011_S_0002,ADNI2,v06,m72,2011-09-19,1,16.0,2,5,80.4682,1.0,0.090,0.002,0.460,0.264,2021-04-14 13:05:55.0
1,2,ADNI_011_S_0002,ADNI2,v21,m96,2013-09-09,1,16.0,2,5,82.4422,1.0,0.288,-0.495,0.460,NaN,2021-04-14 13:05:56.0
2,2,ADNI_011_S_0002,ADNI1,m36,m36,2008-08-27,1,16.0,2,5,77.4073,1.0,0.344,0.419,0.535,NaN,2021-04-14 13:05:56.0
3,2,ADNI_011_S_0002,ADNI2,v41,m120,2015-09-22,1,16.0,2,5,84.4764,1.0,0.382,-0.549,0.595,-0.333,2021-04-14 13:05:56.0
4,2,ADNI_011_S_0002,ADNIGO,m60,m60,2010-09-22,1,16.0,2,5,79.4771,1.0,0.101,0.066,0.620,0.264,2021-04-14 13:05:56.0


In [34]:
diag["Subject"].value_counts()

Subject
123_S_0072     4
128_S_0272     4
941_S_1195     4
003_S_0981     4
141_S_1378     4
              ..
100_S_5036     1
016_S_0991     1
100_S_4511     1
016_S_10071    1
941_S_6607     1
Name: count, Length: 2623, dtype: int64

In [35]:
comb = pd.read_csv("DXSUM_PDXCONV_17Apr2024.csv")[["RID", "PTID" , "PHASE"]]
comb = comb.rename(columns={"PHASE": "Phase"})
comb.info
comb.head()

,RID,PTID,Phase
0,2,011_S_0002,ADNI1
1,3,011_S_0003,ADNI1
2,5,011_S_0005,ADNI1
3,8,011_S_0008,ADNI1
4,7,022_S_0007,ADNI1


In [36]:
m = comb.merge(demo, on = ["RID", "Phase"]).merge(neuro,on = ["RID", "Phase"]).merge(clinical,on = ["RID", "Phase"]).drop_duplicates()

In [37]:
m.columns = [c[:-2] if str(c).endswith(('_x','_y')) else c for c in m.columns]

m = m.loc[:,~m.columns.duplicated()]

In [38]:
diag = diag.rename(columns = {"Subject": "PTID"})

In [39]:
m = m.merge(diag, on = ["PTID", "Phase"])

In [40]:
m["PTID"].value_counts()

PTID
067_S_4767    18
141_S_0697    15
094_S_1417    14
067_S_4072    14
011_S_0023    14
              ..
018_S_5240     1
127_S_5233     1
109_S_5254     1
131_S_0436     1
131_S_6153     1
Name: count, Length: 2232, dtype: int64

In [48]:
m.drop(["ID",  "SITEID", "VISCODE", "VISCODE2", "USERDATE","update_stamp",  "PTSOURCE","DX"], axis=1,inplace=True) 

In [49]:
m.columns

Index(['RID', 'PTID', 'Phase', 'VISDATE', 'PTGENDER', 'PTDOB', 'PTDOBYY',
       'PTHAND', 'PTMARRY', 'PTEDUCAT', 'PTNOTRT', 'PTRTYR', 'PTHOME',
       'PTTLANG', 'PTPLANG', 'PTCOGBEG', 'PTADDX', 'PTETHCAT', 'PTRACCAT',
       'NXVISUAL', 'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR',
       'NXFINGER', 'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT',
       'NXOTHER', 'NXABNORM', 'SUBJECT_KEY', 'EXAMDATE', 'AGE', 'PHC_MEM',
       'PHC_EXF', 'PHC_LAN', 'PHC_VSP', 'Group'],
      dtype='object')

In [50]:
m = m.fillna(-4)
m = m.replace("-4", -4)
percentage_missing = (m == -4).sum() / len(m)

cols_to_delete = m.columns[(percentage_missing > 0.70) & (m.columns)]

m.drop(cols_to_delete, axis = 1, inplace = True)

In [51]:
m.columns

Index(['RID', 'PTID', 'Phase', 'VISDATE', 'PTGENDER', 'PTDOB', 'PTDOBYY',
       'PTHAND', 'PTMARRY', 'PTEDUCAT', 'PTNOTRT', 'PTRTYR', 'PTHOME',
       'PTTLANG', 'PTPLANG', 'PTCOGBEG', 'PTADDX', 'PTETHCAT', 'PTRACCAT',
       'NXVISUAL', 'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR',
       'NXFINGER', 'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT',
       'NXOTHER', 'NXABNORM', 'SUBJECT_KEY', 'EXAMDATE', 'AGE', 'PHC_MEM',
       'PHC_EXF', 'PHC_LAN', 'PHC_VSP', 'Group'],
      dtype='object')

In [52]:
len(m.columns)

41

In [53]:
categorical = ['PTGENDER', 'PTHOME','PTMARRY','PTEDUCAT','PTPLANG','NXVISUAL','PTNOTRT','NXTREMOR','NXAUDITO','PTHAND']

In [54]:
quant = ['PTDOBYY','PHC_MEM','PHC_EXF','PTRACCAT','AGE','PTADDX','PTETHCAT', 'PTCOGBEG','PHC_VSP','PHC_LAN']

In [55]:
text = ["CMMED"]

In [56]:
cols_left = list(set(m.columns) - set(categorical) - set(text)  - set(["label", "Group","GROUP", "Phase", "RID", "PTID"]))
m[cols_left]

,PHC_EXF,PHC_LAN,PTCOGBEG,AGE,NXTENDON,SUBJECT_KEY,PTRTYR,NXHEEL,PTDOBYY,EXAMDATE,...,NXPLANTA,NXGAIT,PTETHCAT,NXABNORM,NXFINGER,PHC_MEM,NXSENSOR,PTRACCAT,PTTLANG,NXCONSCI
0,-1.036,-0.828,-4.0,82.3655,1.0,G-ADNI-AN000439,1989.0,1.0,1924.0,2006-09-12,...,1.0,1.0,2.0,1.0,1.0,-0.973,1.0,5,1.0,1.0
1,-0.670,-0.516,-4.0,81.3662,1.0,G-ADNI-AN000439,1989.0,1.0,1924.0,2005-09-12,...,1.0,1.0,2.0,1.0,1.0,-1.068,1.0,5,1.0,1.0
2,-0.645,-0.756,-4.0,83.3648,1.0,G-ADNI-AN000439,1989.0,1.0,1924.0,2007-09-12,...,1.0,1.0,2.0,1.0,1.0,-1.559,1.0,5,1.0,1.0
3,-0.145,-0.516,-4.0,81.8645,1.0,G-ADNI-AN000439,1989.0,1.0,1924.0,2006-03-13,...,1.0,1.0,2.0,1.0,1.0,-0.768,1.0,5,1.0,1.0
4,0.527,0.034,-4.0,75.7673,1.0,G-ADNI-AN000663,1987.0,1.0,1931.0,2007-09-07,...,1.0,1.0,2.0,1.0,1.0,0.465,2.0,5,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6778,-4.000,-4.000,2017.0,80.9801,1.0,-4,2009.0,1.0,1938.0,2019-04-24,...,1.0,1.0,2.0,1.0,1.0,0.624,1.0,5,1.0,1.0
6779,-4.000,-4.000,2009.0,76.5996,2.0,-4,2009.0,1.0,1943.0,2020-02-05,...,1.0,2.0,2.0,1.0,1.0,-0.180,1.0,5,1.0,1.0
6780,-4.000,-4.000,2014.0,74.7598,-4.0,-4,2019.0,-4.0,1945.0,2019-12-04,...,-4.0,-4.0,2.0,2.0,-4.0,0.870,-4.0,5,1.0,-4.0
6781,-4.000,-4.000,2019.0,74.6804,1.0,-4,2007.0,1.0,1945.0,2020-02-05,...,1.0,1.0,2.0,1.0,2.0,-0.814,1.0,5,1.0,1.0


In [58]:
for col in cols_left:
    if len(m[col].value_counts()) < 10:
        print(col)
        categorical.append(col)

NXTENDON
NXHEEL
NXOTHER
NXMOTOR
NXNERVE
NXPLANTA
NXGAIT
PTETHCAT
NXABNORM
NXFINGER
NXSENSOR
PTRACCAT
PTTLANG
NXCONSCI


In [60]:
to_del = ["PTRTYR", "EXAMDATE", "SUBJECT_KEY"]
m = m.drop(to_del, axis=1)

In [62]:
quant = list(set(cols_left) - set(categorical) - set(text)  -set(to_del) - set(["label", "Group","GROUP", "Phase", "RID", "PTID"]))
m[quant]

,PHC_EXF,PHC_LAN,PTCOGBEG,AGE,PHC_VSP,PTDOBYY,PHC_MEM,VISDATE,PTDOB,PTADDX
0,-1.036,-0.828,-4.0,82.3655,0.128,1924.0,-0.973,2005-08-18,05/1924,-4.0
1,-0.670,-0.516,-4.0,81.3662,-0.237,1924.0,-1.068,2005-08-18,05/1924,-4.0
2,-0.645,-0.756,-4.0,83.3648,-4.000,1924.0,-1.559,2005-08-18,05/1924,-4.0
3,-0.145,-0.516,-4.0,81.8645,-0.305,1924.0,-0.768,2005-08-18,05/1924,-4.0
4,0.527,0.034,-4.0,75.7673,0.258,1931.0,0.465,2005-08-23,12/1931,-4.0
...,...,...,...,...,...,...,...,...,...,...
6778,-4.000,-4.000,2017.0,80.9801,-4.000,1938.0,0.624,2019-04-24,05/1938,9999.0
6779,-4.000,-4.000,2009.0,76.5996,-4.000,1943.0,-0.180,2020-02-05,07/1943,9999.0
6780,-4.000,-4.000,2014.0,74.7598,-4.000,1945.0,0.870,2019-12-04,03/1945,-4.0
6781,-4.000,-4.000,2019.0,74.6804,-4.000,1945.0,-0.814,2020-02-05,06/1945,2019.0


In [66]:
#after reviewing the meaning of each column, these are the final ones
l = ['RID', 'PTID', 'Group', 'Phase', 'PTGENDER', 'PTDOBYY', 'PTHAND',
       'PTMARRY', 'PTEDUCAT', 'PTNOTRT', 'PTHOME', 'PTTLANG',
       'PTPLANG', 'PTCOGBEG', 'PTETHCAT', 'PTRACCAT', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 
       'NXABNORM',  'PHC_MEM', 'PHC_EXF', 'PHC_LAN', 'PHC_VSP']

In [67]:
len(l)

33

In [68]:
m[l]

,RID,PTID,Group,Phase,PTGENDER,PTDOBYY,PTHAND,PTMARRY,PTEDUCAT,PTNOTRT,...,NXHEEL,NXSENSOR,NXTENDON,NXPLANTA,NXGAIT,NXABNORM,PHC_MEM,PHC_EXF,PHC_LAN,PHC_VSP
0,3,011_S_0003,2.0,ADNI1,1.0,1924.0,1.0,1.0,18.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,-0.973,-1.036,-0.828,0.128
1,3,011_S_0003,2.0,ADNI1,1.0,1924.0,1.0,1.0,18.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,-1.068,-0.670,-0.516,-0.237
2,3,011_S_0003,2.0,ADNI1,1.0,1924.0,1.0,1.0,18.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,-1.559,-0.645,-0.756,-4.000
3,3,011_S_0003,2.0,ADNI1,1.0,1924.0,1.0,1.0,18.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,-0.768,-0.145,-0.516,-0.305
4,5,011_S_0005,0.0,ADNI1,1.0,1931.0,1.0,1.0,16.0,1.0,...,1.0,2.0,1.0,1.0,1.0,1.0,0.465,0.527,0.034,0.258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6778,6725,013_S_6725,1.0,ADNI3,1.0,1938.0,1.0,3.0,20.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.624,-4.000,-4.000,-4.000
6779,6851,168_S_6851,1.0,ADNI3,2.0,1943.0,1.0,3.0,18.0,1.0,...,1.0,1.0,2.0,1.0,2.0,1.0,-0.180,-4.000,-4.000,-4.000
6780,6841,035_S_6841,0.0,ADNI3,2.0,1945.0,1.0,1.0,14.0,1.0,...,-4.0,-4.0,-4.0,-4.0,-4.0,2.0,0.870,-4.000,-4.000,-4.000
6781,6855,032_S_6855,2.0,ADNI3,1.0,1945.0,1.0,1.0,16.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,-0.814,-4.000,-4.000,-4.000


In [69]:
dfs=[]

In [71]:
for col in categorical:
    dfs.append(pd.get_dummies(m[col], prefix = col))

In [72]:
dfs

[      PTGENDER_-4.0  PTGENDER_1.0  PTGENDER_2.0
 0             False          True         False
 1             False          True         False
 2             False          True         False
 3             False          True         False
 4             False          True         False
 ...             ...           ...           ...
 6778          False          True         False
 6779          False         False          True
 6780          False         False          True
 6781          False          True         False
 6782          False         False          True
 
 [6783 rows x 3 columns],
       PTHOME_-4.0  PTHOME_1.0  PTHOME_2.0  PTHOME_3.0  PTHOME_4.0  PTHOME_5.0  \
 0           False       False       False        True       False       False   
 1           False       False       False        True       False       False   
 2           False       False       False        True       False       False   
 3           False       False       False        True  

In [73]:
cat = pd.concat(dfs,axis=1)

In [74]:
cat

,PTGENDER_-4.0,PTGENDER_1.0,PTGENDER_2.0,PTHOME_-4.0,PTHOME_1.0,PTHOME_2.0,PTHOME_3.0,PTHOME_4.0,PTHOME_5.0,PTHOME_6.0,...,PTRACCAT_4,PTRACCAT_5,PTRACCAT_6,PTRACCAT_7,PTTLANG_-4.0,PTTLANG_1.0,PTTLANG_2.0,NXCONSCI_-4.0,NXCONSCI_1.0,NXCONSCI_2.0
0,False,True,False,False,False,False,True,False,False,False,...,False,True,False,False,False,True,False,False,True,False
1,False,True,False,False,False,False,True,False,False,False,...,False,True,False,False,False,True,False,False,True,False
2,False,True,False,False,False,False,True,False,False,False,...,False,True,False,False,False,True,False,False,True,False
3,False,True,False,False,False,False,True,False,False,False,...,False,True,False,False,False,True,False,False,True,False
4,False,True,False,False,True,False,False,False,False,False,...,False,True,False,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6778,False,True,False,False,False,False,False,False,False,True,...,False,True,False,False,False,True,False,False,True,False
6779,False,False,True,False,True,False,False,False,False,False,...,False,True,False,False,False,True,False,False,True,False
6780,False,False,True,False,False,False,False,True,False,False,...,False,True,False,False,False,True,False,True,False,False
6781,False,True,False,False,True,False,False,False,False,False,...,False,True,False,False,False,True,False,False,True,False


In [75]:
m[quant]

,PHC_EXF,PHC_LAN,PTCOGBEG,AGE,PHC_VSP,PTDOBYY,PHC_MEM,VISDATE,PTDOB,PTADDX
0,-1.036,-0.828,-4.0,82.3655,0.128,1924.0,-0.973,2005-08-18,05/1924,-4.0
1,-0.670,-0.516,-4.0,81.3662,-0.237,1924.0,-1.068,2005-08-18,05/1924,-4.0
2,-0.645,-0.756,-4.0,83.3648,-4.000,1924.0,-1.559,2005-08-18,05/1924,-4.0
3,-0.145,-0.516,-4.0,81.8645,-0.305,1924.0,-0.768,2005-08-18,05/1924,-4.0
4,0.527,0.034,-4.0,75.7673,0.258,1931.0,0.465,2005-08-23,12/1931,-4.0
...,...,...,...,...,...,...,...,...,...,...
6778,-4.000,-4.000,2017.0,80.9801,-4.000,1938.0,0.624,2019-04-24,05/1938,9999.0
6779,-4.000,-4.000,2009.0,76.5996,-4.000,1943.0,-0.180,2020-02-05,07/1943,9999.0
6780,-4.000,-4.000,2014.0,74.7598,-4.000,1945.0,0.870,2019-12-04,03/1945,-4.0
6781,-4.000,-4.000,2019.0,74.6804,-4.000,1945.0,-0.814,2020-02-05,06/1945,2019.0


In [76]:
m[["PTID","RID", "Phase", "Group"]]

,PTID,RID,Phase,Group
0,011_S_0003,3,ADNI1,2.0
1,011_S_0003,3,ADNI1,2.0
2,011_S_0003,3,ADNI1,2.0
3,011_S_0003,3,ADNI1,2.0
4,011_S_0005,5,ADNI1,0.0
...,...,...,...,...
6778,013_S_6725,6725,ADNI3,1.0
6779,168_S_6851,6851,ADNI3,1.0
6780,035_S_6841,6841,ADNI3,0.0
6781,032_S_6855,6855,ADNI3,2.0


In [79]:
c = pd.concat([m[["PTID", "RID", "Phase", "Group"]].reset_index(), cat.reset_index(), m[quant].reset_index()], axis=1).drop("index", axis=1) 

In [80]:
c

,PTID,RID,Phase,Group,PTGENDER_-4.0,PTGENDER_1.0,PTGENDER_2.0,PTHOME_-4.0,PTHOME_1.0,PTHOME_2.0,...,PHC_EXF,PHC_LAN,PTCOGBEG,AGE,PHC_VSP,PTDOBYY,PHC_MEM,VISDATE,PTDOB,PTADDX
0,011_S_0003,3,ADNI1,2.0,False,True,False,False,False,False,...,-1.036,-0.828,-4.0,82.3655,0.128,1924.0,-0.973,2005-08-18,05/1924,-4.0
1,011_S_0003,3,ADNI1,2.0,False,True,False,False,False,False,...,-0.670,-0.516,-4.0,81.3662,-0.237,1924.0,-1.068,2005-08-18,05/1924,-4.0
2,011_S_0003,3,ADNI1,2.0,False,True,False,False,False,False,...,-0.645,-0.756,-4.0,83.3648,-4.000,1924.0,-1.559,2005-08-18,05/1924,-4.0
3,011_S_0003,3,ADNI1,2.0,False,True,False,False,False,False,...,-0.145,-0.516,-4.0,81.8645,-0.305,1924.0,-0.768,2005-08-18,05/1924,-4.0
4,011_S_0005,5,ADNI1,0.0,False,True,False,False,True,False,...,0.527,0.034,-4.0,75.7673,0.258,1931.0,0.465,2005-08-23,12/1931,-4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6778,013_S_6725,6725,ADNI3,1.0,False,True,False,False,False,False,...,-4.000,-4.000,2017.0,80.9801,-4.000,1938.0,0.624,2019-04-24,05/1938,9999.0
6779,168_S_6851,6851,ADNI3,1.0,False,False,True,False,True,False,...,-4.000,-4.000,2009.0,76.5996,-4.000,1943.0,-0.180,2020-02-05,07/1943,9999.0
6780,035_S_6841,6841,ADNI3,0.0,False,False,True,False,False,False,...,-4.000,-4.000,2014.0,74.7598,-4.000,1945.0,0.870,2019-12-04,03/1945,-4.0
6781,032_S_6855,6855,ADNI3,2.0,False,True,False,False,True,False,...,-4.000,-4.000,2019.0,74.6804,-4.000,1945.0,-0.814,2020-02-05,06/1945,2019.0


In [81]:
c = c.groupby('PTID', group_keys=False,as_index=False).apply(lambda x: x.loc[x["Group"].astype(int).idxmax()]).reset_index(inplace=False)


C:\Users\Adya2\AppData\Local\Temp\ipykernel_28532\1067821708.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  c = c.groupby('PTID', group_keys=False,as_index=False).apply(lambda x: x.loc[x["Group"].astype(int).idxmax()]).reset_index(inplace=False)


In [82]:
c

,index,PTID,RID,Phase,Group,PTGENDER_-4.0,PTGENDER_1.0,PTGENDER_2.0,PTHOME_-4.0,PTHOME_1.0,...,PHC_EXF,PHC_LAN,PTCOGBEG,AGE,PHC_VSP,PTDOBYY,PHC_MEM,VISDATE,PTDOB,PTADDX
0,0,002_S_0295,295,ADNI1,0.0,False,True,False,False,True,...,0.554,0.272,-4.0,84.9363,0.483,1921.0,1.357,2006-04-04,06/1921,-4.0
1,1,002_S_0413,413,ADNI1,0.0,False,False,True,False,True,...,1.480,2.594,-4.0,76.9528,-4.000,1929.0,1.287,2006-04-06,12/1929,-4.0
2,2,002_S_0559,559,ADNI1,0.0,False,True,False,False,True,...,1.176,1.041,-4.0,80.5284,-4.000,1927.0,0.768,2006-05-11,01/1927,-4.0
3,3,002_S_0619,619,ADNI1,2.0,False,True,False,False,False,...,-0.509,0.116,-4.0,78.0315,0.264,1928.0,-1.259,2006-05-18,12/1928,-4.0
4,4,002_S_0685,685,ADNI1,0.0,False,False,True,False,True,...,0.111,1.170,-4.0,92.7228,-4.000,1916.0,0.149,2006-06-22,11/1916,-4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2227,2227,941_S_6580,6580,ADNI3,0.0,False,False,True,False,False,...,-0.208,0.159,9999.0,79.9535,-0.583,1938.0,0.082,2018-08-31,10/1938,9999.0
2228,2228,941_S_6581,6581,ADNI3,0.0,False,False,True,False,False,...,0.824,1.281,9999.0,74.5489,-4.000,1944.0,1.142,2018-09-05,03/1944,9999.0
2229,2229,941_S_6607,6607,ADNI3,0.0,False,True,False,False,True,...,-4.000,-4.000,9999.0,82.7871,-4.000,1936.0,0.624,2018-10-15,01/1936,9999.0
2230,2230,941_S_6803,6803,ADNI3,1.0,False,False,True,False,True,...,-0.854,-0.147,2014.0,75.1266,-0.041,1944.0,-0.604,2019-08-27,08/1944,9999.0


In [83]:
c.to_csv("clinical.csv")

STOP

In [ ]:
m.update(m5[~m5.index.duplicated(keep='first')])
indexes = m.index
m["GROUP"] = -1

In [24]:
for i in indexes:
    row = m.loc[i]
    if (row["GroupN"] == row["Group"]):
        val = row["GroupN"]
        
        m.loc[i, "GROUP"] = val
    elif (row["GroupN"] == row["DX"]):
        val = row["GroupN"]
        m.loc[i, "GROUP"] = val
        
    elif (row["Group"] == row["DX"]):
        val = row["Group"]
        m.loc[i, "GROUP"] = val
        

In [25]:
m5 = m5[~m5.index.duplicated(keep='first')]
m5

,Subject,GroupN,Group,DX,Phase
0,002_S_0295,0.0,0.0,0.0,ADNI1
72,002_S_0295,0.0,0.0,0.0,ADNI2
80,002_S_0413,0.0,0.0,0.0,ADNI1
188,002_S_0413,0.0,0.0,0.0,ADNI2
263,002_S_0413,0.0,0.0,0.0,ADNI3
...,...,...,...,...,...
78575,941_S_6044,2.0,0.0,0.0,ADNI3
78603,941_S_6068,2.0,1.0,1.0,ADNI3
78621,941_S_6080,1.0,0.0,0.0,ADNI3
78657,941_S_6345,2.0,1.0,1.0,ADNI3


In [26]:
m[m["GROUP"] != -1]

,Subject,GroupN,Group,DX,Phase,GROUP
0,002_S_0295,0.0,0.0,0.0,ADNI1,0
72,002_S_0295,0.0,0.0,0.0,ADNI2,0
80,002_S_0413,0.0,0.0,0.0,ADNI1,0
188,002_S_0413,0.0,0.0,0.0,ADNI2,0
263,002_S_0413,0.0,0.0,0.0,ADNI3,0
...,...,...,...,...,...,...
78739,941_S_7074,0.0,0.0,0.0,NaN,0
78741,941_S_7085,1.0,1.0,1.0,NaN,1
78742,941_S_7087,0.0,0.0,0.0,NaN,0
78743,941_S_7091,0.0,0.0,0.0,NaN,0


In [27]:
m[["Subject", "GroupN", "Group", "DX", "GROUP", "Phase"]].to_csv("diagnosis_full.csv")